Here are some inception models I attempted to create but had low accuracy and/or was overfitting.

In [0]:
#For Google Collab
from google.colab import drive, auth
from googleapiclient.discovery import build
#For data manipulation
import numpy as np 
import pandas as pd 
import random
#For Plotting
import matplotlib.pyplot as plt
#For Modeling
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization, Input, concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers, regularizers
from keras.optimizers import Adam, SGD

In [0]:
# Load the Drive helper and mount
# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
auth.authenticate_user()

In [0]:
drive_service = build('drive', 'v3')

In [0]:
#Create variable for image and batch size along with number of epochs committing to for model
height = 224
width = 224
batch_size = 32

train = ImageDataGenerator(rescale = 1/.255,rotation_range=30,horizontal_flip=True)
test = ImageDataGenerator(rescale=1/.255,rotation_range=30,horizontal_flip=True)

In [0]:
#Seting up train, test, and val sets for model

train_set = train.flow_from_directory('../content/drive/My Drive/Chest_X_rays/All_Chest/train'
                                      ,target_size=(width,height), 
                                      batch_size=batch_size,
                                      class_mode='categorical',              
                                     seed = 122)
test_set = train.flow_from_directory('../content/drive/My Drive/Chest_X_rays/All_Chest/test'
                                      ,target_size=(width,height), 
                                      batch_size=batch_size,
                                      class_mode='categorical',                              
                                    seed = 122)
val_set = train.flow_from_directory('../content/drive/My Drive/Chest_X_rays/All_Chest/val'
                                      ,target_size=(width,height), 
                                      batch_size=batch_size,
                                      class_mode='categorical',
                                    seed = 122)

In [0]:
inputs = Input(shape=(height, width, 3))
tower1_1 = MaxPooling2D((3,3), strides = (1,1), padding = 'same')(inputs)
tower1_1 = Conv2D(32, (1,1), activation='relu', padding='same')(tower1_1)
tower1_2 = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
tower1_3 = Conv2D(32, (5,5), activation='relu', padding='same')(inputs)

output = keras.layers.concatenate([tower1_1, tower1_2, tower1_3], axis = 3)
output = Flatten()(output)
out    = Dense(3, activation='softmax')(output)

model = Model(inputs = inputs, outputs = out)

In [0]:
def model1(train_set):
  x = inception1(inputs)
  x = Flatten()(x)
  x = Dense(32, activation='relu')(x)
  predictions = Dense(3, activation='softmax')(x)

  model = Model(input=inputs, output=predictions)
  return model

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
                 optimizer=Adam(lr=0.0001),
                 metrics=['accuracy'])
rt=ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.01, cooldown=1, min_lr=.000000001)
history = model.fit_generator(train_set, steps_per_epoch=25, epochs=10, validation_steps=50, verbose=1, validation_data=test_set, callbacks = [rt])
history

In [0]:
plt.plot(history.history['loss'], label = 'Train')
plt.plot(history.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history.history['acc'], label = 'Train')
plt.plot(history.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()

In [0]:
#Second best Inception model
def inception3(inputs):
  tower_one = MaxPooling2D((1,1), strides=(1,1), padding='same')(inputs)
  tower_one = Conv2D(32, (1,1), activation='relu', padding='same')(tower_one)

  tower_two = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
  tower_two = Conv2D(32, (3,3), activation='relu', padding='same')(tower_two)
  
  tower_three = BatchNormalization()(inputs)
  tower_three = Conv2D(32, (3,3), activation='relu', padding='same')(tower_three)

  tower_four = Conv2D(32, (5,5), activation='relu', padding='same')(inputs)
  tower_four = Conv2D(32, (5,5), activation='relu', padding='same')(tower_four)
  
  tower_five = BatchNormalization()(inputs)
  tower_five = Conv2D(32, (5,5), activation='relu', padding='same')(tower_five)
  
  x = concatenate([tower_one, tower_two, tower_three, tower_four, tower_five], axis=3)

  return x

In [0]:
def model3(train_set):
  x = inception3(inputs)
  x = MaxPooling2D((2, 2))(x)
  x = BatchNormalization()(x)
  x = inception3(x)
  x = MaxPooling2D((2, 2))(x)
  x = BatchNormalization()(x)
  x = Flatten()(x)
  x = Dense(128, activation='relu')(x)
  predictions = Dense(3, activation='softmax')(x)

  model = Model(input=inputs, output=predictions)
  return model

modelC = model3(train_set)

In [0]:
modelC.compile(loss=keras.losses.categorical_crossentropy,
                 optimizer=Adam(lr=0.0001),
                 metrics=['accuracy'])
history = modelC.fit_generator(train_set, steps_per_epoch=10, epochs=10, validation_steps=50, verbose=1, validation_data=test_set)
history

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
                 optimizer=Adam(lr=0.001),
                 metrics=['accuracy'])
rt=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='auto', min_delta=0.01, cooldown=1, min_lr=.000000001)
history = model.fit_generator(train_set, steps_per_epoch=20, epochs=5, validation_steps=50, verbose=1, validation_data=test_set, callbacks = [rt])
history

In [0]:
def inception3(inputs):
  tower_one = MaxPooling2D((1,1), strides=(1,1), padding='same')(inputs)
  tower_one = Conv2D(32, (1,1),kernel_regularizer=regularizers.l2(0.1), activation='relu', padding='same')(tower_one)

  tower_two = Conv2D(32, (1,1), activation='relu', padding='same')(inputs)
  tower_two = Conv2D(32, (1,1), activation='relu', padding='same')(tower_two)
  tower_two = Dropout(0.1)(tower_two)
  
  tower_three = BatchNormalization()(inputs)
  tower_three = Conv2D(32, (3,3),kernel_regularizer=regularizers.l2(0.1), activation='relu', padding='same')(tower_three)

  tower_four = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
  tower_four = Conv2D(32, (3,3), activation='relu', padding='same')(tower_four)
  tower_four = Dropout(.1)(tower_four)
  
  tower_five = Conv2D(32, (5,5), activation='relu', padding='same')(inputs)
  tower_five = Conv2D(32, (5,5), activation='relu', padding='same')(tower_five)
  tower_five = Dropout(.1)(tower_five)
  
  tower_six = BatchNormalization()(inputs)
  tower_six = Conv2D(32, (5,5),kernel_regularizer=regularizers.l2(0.1), activation='relu', padding='same')(tower_six)
  
  x = concatenate([tower_one, tower_two, tower_three, tower_four, tower_five, tower_six], axis=3)

  return x

In [0]:
def model3(train_set):
  x = inception3(inputs)
  x = MaxPooling2D((2, 2))(x)
  x = inception3(x)
  x = Dropout(.2)(x)
  x = inception3(x)
  x = BatchNormalization()(x)
  x = inception3(x)
  x = Dropout(.1)(x)
  x = inception3(x)
  x = BatchNormalization()(x)
  x = Dropout(.1)(x)
  x = inception3(x)
  x = MaxPooling2D((2, 2))(x)
  x = Flatten()(x)
  x = Dense(128, activation='relu')(x)
  predictions = Dense(3, activation='softmax')(x)

  model = Model(input=inputs, output=predictions)

  return model

modelC = model3(train_set)

In [0]:
modelC.compile(loss=keras.losses.categorical_crossentropy,
                 optimizer=Adam(lr=0.00005),
                 metrics=['accuracy'])
history = modelC.fit_generator(train_set, steps_per_epoch=25, epochs=10, validation_steps=15, verbose=1, validation_data=test_set)
history

In [0]:
plt.plot(history.history['loss'], label = 'Train')
plt.plot(history.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history.history['acc'], label = 'Train')
plt.plot(history.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()